## Sentiment analysis
-----
### Importing the cleaned data

In [66]:
%store -r game_df_cleaned
%store -r swords_df_cleaned

%store -r game_text_cleaned
%store -r swords_text_cleaned

In [24]:
!pip install afinn

In [25]:
from afinn import Afinn as Afinn
import re
from siuba import *
import pandas as pd
# from wordcloud import *

In [84]:
afinn = Afinn()

game_text_str = ""
for _list in game_text_cleaned:
    for string in _list:
        game_text_str += string + ' '

game_df_cleaned = pd.DataFrame({
    "word": game_text_str.split()
})

swords_text_str = ""
for _list in swords_text_cleaned:
    for string in _list:
        swords_text_str += string + ' '

swords_df_cleaned = pd.DataFrame({
    "word": swords_text_str.split()
})

word_scores_swords = [afinn.score(word_) for word_ in swords_df_cleaned['word']]
swords_df_cleaned = swords_df_cleaned.assign(word_score = word_scores_swords)
print(swords_df_cleaned)

word_scores_game = [afinn.score(word) for word in game_df_cleaned['word']]
game_df_cleaned = game_df_cleaned.assign(word_score = word_scores_game)
print(game_df_cleaned)

            word  word_score
0          STORM         0.0
1             OF         0.0
2         SWORDS         0.0
3           Book         0.0
4           Song         0.0
...          ...         ...
227179     great         3.0
227180  nineteen         0.0
227181  previous         0.0
227182     Table         0.0
227183  Contents         0.0

[227184 rows x 2 columns]
            word  word_score
0           GAME         0.0
1             OF         0.0
2        THRONES         0.0
3           Book         0.0
4           Song         0.0
...          ...         ...
279431      maid         0.0
279432  thirteen         0.0
279433  previous         0.0
279434     Table         0.0
279435  Contents         0.0

[279436 rows x 2 columns]


### Function to break first book into chapters
------
#### Chapter 0 ----> PROLOGUE

In [80]:
chapters_first_book = ["BRAN", "CATELYN", "DAENERYS", "EDDARD", "JON", "ARYA", "TYRION", "SANSA"]
chapters_second_book = ["JAIME", "DAVOS", "SAMWELL", "BRAN", "CATELYN", "DAENERYS", "JON", "ARYA", "TYRION", "SANSA"]

# CHAPTER 0  -------> PROLOGUE
def select_chapter(dataframe, chapters):
    chapter_list = []
    curr_chapter = 0
    flag = False
    flag_epilogue = 0
    for index, row in dataframe.iterrows():
        if flag:
            curr_chapter = curr_chapter
        elif re.search("HOUSE", row['word']) and "SAMWELL" not in chapters:
            flag = True
        elif re.search("EPILOGUE", row['word']):
            flag_epilogue += 1
            if flag_epilogue > 1:
                flag = True
        elif row['word'] in chapters:
            curr_chapter += 1
        chapter_list.append(curr_chapter)
    return chapter_list

### Testing the above function
-----

In [85]:
game_df_cleaned = game_df_cleaned.assign(chapter = select_chapter(game_df_cleaned, chapters_first_book))
print(game_df_cleaned)

swords_df_cleaned = swords_df_cleaned.assign(chapter = select_chapter(swords_df_cleaned, chapters_second_book))
print(swords_df_cleaned)


            word  word_score  chapter
0           GAME         0.0        0
1             OF         0.0        0
2        THRONES         0.0        0
3           Book         0.0        0
4           Song         0.0        0
...          ...         ...      ...
279431      maid         0.0       72
279432  thirteen         0.0       72
279433  previous         0.0       72
279434     Table         0.0       72
279435  Contents         0.0       72

[279436 rows x 3 columns]
            word  word_score  chapter
0          STORM         0.0        0
1             OF         0.0        0
2         SWORDS         0.0        0
3           Book         0.0        0
4           Song         0.0        0
...          ...         ...      ...
227179     great         3.0       80
227180  nineteen         0.0       80
227181  previous         0.0       80
227182     Table         0.0       80
227183  Contents         0.0       80

[227184 rows x 3 columns]


In [87]:
swords_df_cleaned_grouped = group_by(swords_df_cleaned, 'chapter')
summarized_ss_df_swords = summarize(swords_df_cleaned_grouped, chapter_score = _.word_score.sum())
print(summarized_ss_df_swords)

game_df_cleaned_grouped = group_by(game_df_cleaned, 'chapter')
summarized_ss_df = summarize(game_df_cleaned_grouped, chapter_score = _.word_score.sum())
print(summarized_ss_df)


    chapter  chapter_score
0         0         -238.0
1         1         -113.0
2         2          -14.0
3         3          -94.0
4         4          -69.0
..      ...            ...
76       76           -4.0
77       77         -128.0
78       78            4.0
79       79          -38.0
80       80        -5608.0

[81 rows x 2 columns]
    chapter  chapter_score
0         0          -56.0
1         1          -90.0
2         2            2.0
3         3            6.0
4         4           -9.0
..      ...            ...
68       68          -45.0
69       69          -19.0
70       70          -89.0
71       71          -16.0
72       72        -7896.0

[73 rows x 2 columns]


In [88]:
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

mask = np.array(Image.open("map.png"))
mask[mask == 0] = 255

chapter_51 = swords_df_cleaned[swords_df_cleaned['chapter'] == 51]
text_str = ""
for str in chapter_51['word']:
    text_str += str + " "

cloudy_cloud_2 = WordCloud(background_color="white", width=800, height=400, colormap='Set2', max_words=500, mask=mask).generate(text_str)

plt.figure( figsize=(20,10), facecolor='k')
plt.imshow(cloudy_cloud_2)
plt.axis("off")
plt.tight_layout(pad=0)

NameError: name 'WordCloud' is not defined